In [23]:
import numpy as np
import pyatomdb
import seaborn as sns
import pandas as pd
from ipywidgets import interact
import ipywidgets as widgets
from mendeleev import element
import matplotlib.pyplot as plt
plt.rcParams["font.size"] = 8
sns.set(font_scale=1)
sns.set_style('ticks')
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.top'] = True
plt.rcParams['ytick.right'] = True

# イオンフラクションの調べ方

PyAtomDBのモジュールの中には`pyatomdb.apec.return_ionbal`というイオンフラクションを返してくれる関数がある。使い方の説明は、[こちら](https://atomdb.readthedocs.io/en/master/apec.html#pyatomdb.apec.return_ionbal)。

>**pyatomdb.apec.return_ionbal(Z, Te, init_pop=False, tau=False, teunit='K', filename=False, datacache=False, fast=True, settings=False, debug=False, extrap=True)**<br>
Solve the ionization balance for a element Z.
- Parameters
  - Z: int<br>
    atomic number of element
  - Te: float or array<br>
    electron temperature(s), default in K
  - init_pop: float array<br>
    initial population of ions for non-equlibrium calculations. Will be renormalised to 1.
  - tau: float or array<br>
    N_e * t for the non-equilibrium ioniziation, in cm^3 s^-1.
  - Te_init: float<br>
    initial ionization balance temperature, same units as Te
  - teunit: {‘K’ , ‘keV’}<br>
    units of temperatures (default K)
  - filename: string<br>
    Can optionally point directly to the file in question, i.e. to look at older data look at $HEADAS/../spectral/modelData/eigenELSYMB_v3.0.fits. If not set, download from AtomDB FTP site.
  - datacache: dict<br>
    Used for caching the data. See description in atomdb.get_data
  - fast: bool<br>
    If true, use precalculated eigenvector files to obtain CIE and NEI results
- Returns
  - final_pop: float array<br>
    final populations.

# 使い方の例
## 電離しているプラズマのイオンフラクション
`init_pop = 'ionizing'` と入力すると、中性状態を始状態としたときのイオンフラクションを計算する。

In [24]:
element_name = 'Fe'  # イオンフラクションを調べたい元素
Z = element(element_name).atomic_number
kTe = 5  # 調べたい温度 (keV)
net = 1e10  # 調べたい電離度 (cm-3 s)
pyatomdb.apec.return_ionbal(Z=Z, Te=kTe, tau=net, init_pop='ionizing', teunit='keV')

array([2.22044605e-16, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 4.08763206e-07, 1.03618626e-05,
       3.32979406e-04, 4.13602900e-03, 3.27930403e-02, 7.56247041e-02,
       3.58839467e-01, 3.42019644e-01, 1.39572247e-01, 3.93208344e-02,
       6.64615456e-03, 6.65918737e-04, 3.71650278e-05, 1.03405223e-06,
       1.17900019e-08, 0.00000000e+00, 0.00000000e+00])

Fe$^{+0}$, Fe$^{+1}$, ... Fe$^{+25}$, Fe$^{+26}$の順でイオンフラクションが表示される。

## 再結合しているプラズマのイオンフラクション
同様に、`init_pop = 'recombining'` と入力すると、完全電離を始状態としたときのイオンフラクションを計算する。

In [25]:
element_name = 'Fe'  # イオンフラクションを調べたい元素
Z = element(element_name).atomic_number
kTe = 5  # 調べたい温度 (keV)
net = 1e10  # 調べたい電離度 (cm-3 s)
pyatomdb.apec.return_ionbal(Z=Z, Te=kTe, tau=net, init_pop='recombining', teunit='keV')

array([-2.22044605e-16,  7.39399243e-13,  1.04776195e-12,  9.48379966e-13,
        2.15130089e-12,  2.95009821e-12,  5.94747444e-12,  9.80738261e-12,
        1.26966981e-11,  1.75668699e-11,  2.54312091e-11,  2.75959398e-11,
        4.00645284e-11,  8.03529060e-11,  3.34302730e-10,  6.85281449e-10,
        3.08438118e-09,  2.74417709e-09,  1.01191671e-09,  2.36910546e-10,
        2.40491983e-11,  1.61863332e-11,  3.71853802e-09,  7.53271400e-07,
        1.76866373e-04,  2.00774421e-02,  9.79744926e-01])

## `init_pop`の定義の仕方

>init_pop can be be defined in a range of ways:
- float<br>
  If it is a single number, this is assumed to be an electron temperature. The ionization fraction will be calculated at this .
- dict<br>
  If a dictionary is provided, it should be an explicit ionization fraction for each element. So dict[6]= [0.0, 0.1, 0.2, 0.3, 0.3, 0.1, 0.0] would be the ionization fraction for carbon, dict[8]= [0.0, 0.0, 0.1, 0.1, 0.1, 0.15,0.3,0.2,0.05] would be for oxygen etc.
- ‘ionizing’<br>
  If the string ionizing is provided, set all elements to be entirely neutral. This is the default.
- ‘recombining’<br>
  If the string recombining is provided, set all elements to be fully ionized.

# 応用編: ion fractionのグラフの描画

これを用いると、Smith et al. 2014 ([Link](https://ui.adsabs.harvard.edu/abs/2014arXiv1412.1172S/abstract)) Figure 9のような図をいろいろな元素で作れます。
![Smith2014](https://drive.google.com/uc?id=1-6AgJWhtqufhjvhanLjODWgmU4yk_8sP)

## 横軸 $kT_e$のグラフ

In [26]:
def RetunIonPopArray_net(net, element_name):
    Z = element(element_name).atomic_number
    kTe_num = 64
    kTe_array = np.logspace(-3, 2, num=kTe_num, base=10.0)
    column_array = [element_name + '+' + str(i) for i in range(Z+1)]
    ionpop_array = pd.DataFrame(
        np.array([pyatomdb.apec.return_ionbal(Z=Z, Te=kTe, tau=net, init_pop='ionizing', teunit='keV') for kTe in kTe_array]),
        columns=column_array,
        index=kTe_array
    )
    ionpop_array.columns.name = 'Ion'
    ionpop_array.index.name = r'$kT_{e}$ (keV)'
    return ionpop_array


def set_topax_avecharge(ion_pop, ax):
    avecharge = np.sum(ion_pop * np.arange(ion_pop.shape[1]), axis=1)
    ax_top = ax.twiny()
    ax_top.set_xlim(ax.get_xlim())
    ax_top.set_xscale('log')
    tick_posi = [False if int(avecharge.values[i]) == int(avecharge.values[i+1]) else True for i in range(0, len(avecharge)-1)]
    tick_posi.append(False)
    ticks = avecharge.index[tick_posi].values
    tick_labels = [int(avecharge.values[i])+1 for i in range(len(avecharge)) if tick_posi[i]]
    ax_top.xaxis.set_ticks(ticks, labels=tick_labels)
    ax_top.tick_params(axis='x', which='major', labelsize=6)
    ax_top.set_xlabel('Average Charge')
    ax_top.minorticks_off()
    return ax_top


def PlotIonFraction_net(element_name, net):
    ionpop = RetunIonPopArray_net(net, element_name)
    _, ax = plt.subplots(figsize=(9, 4))
    ax = sns.lineplot(data=ionpop, ax=ax,  linewidth=1.5)
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_ylabel('Ion Fraction')
    ax.set(xlim=(1e-3,1e2), ylim=(0.01,1.0))
    ax.legend(bbox_to_anchor=(1.01, 0.0), loc='lower left', borderaxespad=0, ncol=2, fontsize=8)
    ax.set_title(f'{element_name} Ion Fraction ($n_et$ = {net:.1e} cm$^{{-3}}$ s)')
    ax_top = set_topax_avecharge(ionpop, ax)


element_list = [element(i+1).symbol for i in range(30)]
element_dropdown = widgets.Dropdown(
    options=element_list,
    value = 'Fe',
    description='Element:',
)
net_textbox = widgets.FloatText(
    value = 1e13,
    description = 'tau (cm-3 s):',
    format = '.1e',
)
interact(PlotIonFraction_net, element_name=element_dropdown, net=net_textbox)

interactive(children=(Dropdown(description='Element:', index=25, options=('H', 'He', 'Li', 'Be', 'B', 'C', 'N'…

<function __main__.PlotIonFraction_net(element_name, net)>

## 横軸 $n_e t$のグラフ

In [27]:
def RetunIonPopArray(kTe, element_name, init_pop):
    Z = element(element_name).atomic_number
    net_num = 64
    net_array = np.logspace(5.0, 13.0, num=net_num, base=10.0)
    column_array = [element_name + '+' + str(i) for i in range(Z+1)]
    ionpop_array = pd.DataFrame(
        np.array([pyatomdb.apec.return_ionbal(Z = Z, Te = kTe, tau=net, init_pop=init_pop, teunit='keV') for net in net_array]),
        columns=column_array,
        index=net_array
    )
    ionpop_array.columns.name = 'Ion'
    ionpop_array.index.name = r'$n_{e}t$ (cm$^{-3}$ s)'
    return ionpop_array


def PlotIonFraction(element_name, kTe, init_pop):
    def is_num(s):
        try:
            float(s)
        except ValueError:
            return s
        else:
            return float(s)
    ionpop = RetunIonPopArray(kTe, element_name, is_num(init_pop))
    _, ax = plt.subplots(figsize=(9, 4))
    sns.lineplot(data=ionpop, ax=ax, linewidth=1.5)
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_ylabel('Ion Fraction')
    ax.set(xlim=(1e5,1e13), ylim=(0.001,1.0))
    ax.legend(bbox_to_anchor=(1.01, 0.0), loc='lower left', borderaxespad=0, ncol=2, fontsize=8)
    ax.set_title(f'{element_name} Ion Fraction ($kT_e$ = {kTe:.1f} keV)')
    ax_top = set_topax_avecharge(ionpop, ax)
    ax_top.set_xlabel('Average Charge')
    ax_top.minorticks_off()


element_list = [element(i+1).symbol for i in range(30)]
element_dropdown = widgets.Dropdown(
    options=element_list,
    value = 'Fe',
    description='Element:',
)
kTe_textbox = widgets.FloatText(
    value = 5.0,
    description='kTe (keV): '
)
initpop_textbox = widgets.Text(
    value = 'ionizing',
    description='init_pop: '
)
interact(PlotIonFraction, element_name=element_dropdown, kTe=kTe_textbox, init_pop=initpop_textbox)

interactive(children=(Dropdown(description='Element:', index=25, options=('H', 'He', 'Li', 'Be', 'B', 'C', 'N'…

<function __main__.PlotIonFraction(element_name, kTe, init_pop)>

In [28]:
ion_pop = RetunIonPopArray(1.0, 'Fe', 'ionizing')
avecharge = np.sum(ion_pop * np.arange(ion_pop.shape[1]), axis=1)
print(avecharge)
# print(avecharge.values)
# print([int(i) for i in avecharge.values])
tick_posi = [False if int(avecharge.values[i]) == int(avecharge.values[i-1]) else True for i in range(1, len(avecharge))]
tick_posi.insert(0, True)
# print(tick_posi)
# print(avecharge[tick_posi])
print(avecharge[tick_posi].index)
print(avecharge.loc[avecharge[tick_posi].index.values].values)
print([int(avecharge.values[i]) for i in range(len(avecharge)) if tick_posi[i]])

$n_{e}t$ (cm$^{-3}$ s)
1.000000e+05     0.015674
1.339628e+05     0.020966
1.794602e+05     0.028029
2.404099e+05     0.037448
3.220598e+05     0.049985
                  ...    
3.105013e+12    19.948073
4.159562e+12    19.948236
5.572265e+12    19.948246
7.464760e+12    19.948247
1.000000e+13    19.948247
Length: 64, dtype: float64
Index([          100000.0, 10758358.985421786,  25864162.05275971,
        46415888.33612773,  83298066.47658272,  111588399.2507748,
        200256813.6043113,  359381366.3804626,  481437242.0784339,
         863988449.483969,  1550515779.832622,   2077113925.96645,
       2782559402.2071257,   4993587893.47314,  6689548786.914153,
         8961505019.46605, 12005080577.484068, 28861404414.300896,
        69385678787.37195, 166810053720.00558],
      dtype='float64', name='$n_{e}t$ (cm$^{-3}$ s)')
[1.56740443e-02 1.19139129e+00 2.23730486e+00 3.27732894e+00
 4.47456761e+00 5.07518494e+00 6.27509970e+00 7.46283647e+00
 8.04636881e+00 9.24311704e+00 1.05775